# Lecture 01 — Finite-Dimensional Analogy (Trivial but Meaningful)\n\nThis notebook illustrates the **reduced formulation** idea on a tiny linear-quadratic problem.\n\nWe consider a constraint $A y = B u$ (invertible $A$) and a quadratic objective\n$$\nJ(y,u) = \tfrac12\|y - y_d\|^2 + \tfrac\alpha2\|u\|^2.\n$$\n\nEliminating $y$ gives $y = S u$ with $S = A^{-1}B$, hence the reduced cost $f(u)=J(Su,u)$.\n

In [ ]:
import numpy as np\nimport matplotlib.pyplot as plt\nfrom pathlib import Path\n\nROOT = Path.cwd().parents[1]\nOUT_DIR = ROOT / "jupyterbook" / "slides" / "assets" / "lecture01"\nOUT_DIR.mkdir(parents=True, exist_ok=True)\n\nnp.random.seed(0)\n

## Build a Tiny Problem\n\nWe'll work with small matrices and verify that the reduced cost is convex and has a unique minimizer.

In [ ]:
n, m = 6, 3\nA = np.random.randn(n, n)\nA = A.T @ A + 0.5 * np.eye(n)  # SPD => invertible\nB = np.random.randn(n, m)\nalpha = 1e-2\ny_d = np.random.randn(n)\n\nS = np.linalg.solve(A, B)\n\ndef J(y, u):\n    return 0.5 * np.dot(y - y_d, y - y_d) + 0.5 * alpha * np.dot(u, u)\n\ndef f(u):\n    y = S @ u\n    return J(y, u)\n

## Closed-Form Minimizer\n\nFor this quadratic reduced problem, the minimizer solves\n$$\n(S^T S + \alpha I)u = S^T y_d.\n$$

In [ ]:
H = S.T @ S + alpha * np.eye(m)\nrhs = S.T @ y_d\nu_star = np.linalg.solve(H, rhs)\ny_star = S @ u_star\n\nf_star = f(u_star)\nf_star\n

## A 1D Slice Plot\n\nPlot $f(u_\* + t d)$ along a random direction $d$.

In [ ]:
d = np.random.randn(m)\nd /= np.linalg.norm(d)\nt = np.linspace(-2.0, 2.0, 300)\nvals = np.array([f(u_star + ti * d) for ti in t])\n\nfig, ax = plt.subplots(figsize=(6.0, 3.5))\nax.plot(t, vals, linewidth=2)\nax.axvline(0.0, linestyle="--", linewidth=1.5)\nax.set_title("Reduced cost along a 1D slice")\nax.set_xlabel("t")\nax.set_ylabel(r"$f(u_* + t d)$")\nax.grid(True, alpha=0.25)\nfig.savefig(OUT_DIR / "reduced_cost_slice.png", dpi=200, bbox_inches="tight")\nplt.close(fig)\n\nOUT_DIR / "reduced_cost_slice.png"\n

## Next Step (What Changes for PDEs)\n\n- $S$ becomes the solution operator of a PDE (still linear in LQ elliptic problems).\n- Gradients are computed efficiently via adjoints (no explicit $S$ matrix).\n